In [ ]:
baseline = 4.3
day_obs_iso = '2033-04-02'

# LSST Progress Mockup

This mockup is intended as a tool for iterating LSST progress report design.

Currently, it does not show results based on any "real" data.
The statistics it plots are currently read from data files update by hand, made using an alternate weather simulation.
For production use, these statistics will be automatically generated from completed visits.

In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
import sys
import os

In [ ]:
at_usdf = 'usdf-rsp' in os.getenv("EXTERNAL_INSTANCE_URL", "")

In [ ]:
if at_usdf:
    sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_scheduler")
    sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_sim")
    sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/schedview")

In [ ]:
import logging
from logging import debug, info, warning
from warnings import warn
import warnings
from pathlib import Path
from tempfile import TemporaryDirectory
from collections import OrderedDict

import astropy.units as u
import astropy.utils.iers
import bokeh
import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting
import colorcet as cc
import healpy as hp
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import panel as pn
from astropy.time import Time

In [ ]:
# If IERS degraded accuracy encountered, don't fail, just keep going.
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

logging.basicConfig(level=logging.WARNING)

warnings.filterwarnings(
    "ignore",
    category=UserWarning,
    message=r"Could not determine site from EXTERNAL_INSTANCE_URL.*",
)

warnings.filterwarnings(
    "ignore",
    append=True,
    message=r".*Tried to get polar motions for times after IERS data is valid.*",
)

warnings.filterwarnings("ignore", append=True, message=r".*dubious year.*")

In [ ]:
import schedview.collect.visits
import schedview.compute.visits
import schedview.plot
from rubin_sim import maf
from schedview import DayObs
from lsst.resources import ResourcePath
from rubin_scheduler.utils import ddf_locations

In [ ]:
bokeh.io.output_notebook(hide_banner=True)

In [ ]:
day_obs = DayObs.from_date(day_obs_iso)

In [ ]:
sim_data_dir = Path("/sdf/group/rubin/web_data/sim-data")
baseline_fname = sim_data_dir.joinpath('sims_featureScheduler_runs4.3/baseline/baseline_v4.3.2_10yrs.db')
visits_fname = sim_data_dir.joinpath('sims_featureScheduler_runs4.3/weather/weather_cloudso1v4.3_10yrs.db')

## Progress as measured by scalar ("summary") metrics

Any MAF summary metric (such os those reported [here](https://usdf-maf.slac.stanford.edu/summaryStats?runId=1)) can potentially be computed and reported on either of these plots.

Two examples are shown below:
 - the coadd depth in g, which shows the advantages plotting the instantaneous metric value vs. date.
 - the total area with more than 750 visits in square degrees (f0Area), an example for which the extrapolated final value is a more useful measure of progress toward the final objective.

In [ ]:
summary_path = Path('/sdf/data/rubin/user/neilsen/data/lsst_progress_mockup')
summary_fname = summary_path.joinpath('progress_bymonth.h5').as_posix()

### Median depth in g

In the left-hand plot below, the points and black line show the metric values at each point in time, as measured only from visits at or before that time.
The green line shows the same metric values measured at these times in the baseline simulation. This kind of a plot is suitable for metric values that inclease steadily over time.

The right-hand plot below shows the final g band depth, extrapolated from the date to the end using the baseline simulation.

In [ ]:
median_depth_g = pd.read_hdf(summary_fname, 'median_depth_g')
median_depth_g.loc[day_obs.mjd:, 'extrapolated'] = np.nan
median_depth_g.loc[day_obs.mjd:, 'snapshot'] = np.nan
median_depth_g['date'] = pd.to_datetime(2400000.5 + median_depth_g.index, unit='D', origin='julian')
median_depth_g_ds = bokeh.models.ColumnDataSource(median_depth_g)

In [ ]:
metric_at_date_fig = bokeh.plotting.figure(title="Median g depth at date", x_axis_label='Date', y_axis_label='Metric value', x_axis_type='datetime')
metric_at_date_fig.line(x='date', y='baseline', color='lightgreen', width=5, source=median_depth_g_ds)
metric_at_date_fig.line(x='date', y='snapshot', color='black', source=median_depth_g_ds)
metric_at_date_fig.scatter(x='date', y='snapshot', color='black', source=median_depth_g_ds)

baseline_final_depth = median_depth_g.loc[median_depth_g.index.max(), 'baseline']
extrapolated_metric_fig = bokeh.plotting.figure(title="Final g depth extrapolated from date to end with baseline", x_axis_label='Date', y_axis_label='Metric value', x_axis_type='datetime')
extrapolated_metric_fig.ray(x=median_depth_g['date'].min(), y=baseline_final_depth, length=0, angle=0, width=5, color='lightgreen')
extrapolated_metric_fig.line(x='date', y='extrapolated', color='black', source=median_depth_g_ds)
extrapolated_metric_fig.scatter(x='date', y='extrapolated', color='black', source=median_depth_g_ds)

metric_figs = bokeh.layouts.row(metric_at_date_fig, extrapolated_metric_fig)
bokeh.io.show(metric_figs)

### Total area with more than 750 visits

For some metrics, the current value does not provide a good indication of progress on that metric. For example, the value of fOArea metric (the total area that has reached a given depth) might not change much even when significant progress is being made if the progress is spread out evenly over the whole footprint.

For these metrics, the plot on the right, which shows the value extrapolated to the end of the survey, is more useful.

This value will be pessimistic, because it does not account for any response the scheduler might make to the difference in completed vs. baseline visits. A different approach is to create bespoke simulations from each date, starting the scheduler simulator with the completed visits pre-loaded and simulating to the end of the survey. The general appearance of the figure would remain much the same, however.

In the plot below, the black line and points show the metric values measured from the chimera simulations, while the green is the value from the baseline.

In [ ]:
f0area = pd.read_hdf(summary_fname, 'fOArea750')
f0area.loc[day_obs.mjd:, 'extrapolated'] = np.nan
f0area.loc[day_obs.mjd:, 'snapshot'] = np.nan
f0area['date'] = pd.to_datetime(2400000.5 + f0area.index, unit='D', origin='julian')
f0area_ds = bokeh.models.ColumnDataSource(f0area)

In [ ]:
metric_at_date_fig = bokeh.plotting.figure(title="Square degrees with more than 750 visits at data", x_axis_label='Date', y_axis_label='Metric value', x_axis_type='datetime')
metric_at_date_fig.line(x='date', y='baseline', color='lightgreen', width=5, source=f0area_ds)
metric_at_date_fig.line(x='date', y='snapshot', color='black', source=f0area_ds)
metric_at_date_fig.scatter(x='date', y='snapshot', color='black', source=f0area_ds)

baseline_final_f0area = f0area.loc[f0area.index.max(), 'baseline']
extrapolated_metric_fig = bokeh.plotting.figure(title="Final area with more than 750 visits extrapolated from date to end with baseline", x_axis_label='Date', y_axis_label='Metric value', x_axis_type='datetime')
extrapolated_metric_fig.ray(x=f0area['date'].min(), y=baseline_final_f0area, length=0, angle=0, width=5, color='lightgreen')
extrapolated_metric_fig.line(x='date', y='extrapolated', color='black', source=f0area_ds)
extrapolated_metric_fig.scatter(x='date', y='extrapolated', color='black', source=f0area_ds)

metric_figs = bokeh.layouts.row(metric_at_date_fig, extrapolated_metric_fig)
bokeh.io.show(metric_figs)

## Hourglass plots

Hourglass plots can instantaneous values over a large period of time, e.g. a year: the colors each horizontal line within a column represent the values over that night. Hourglass plots can be additionally annotated with twilight times, moon rise, transit, and set times, and other astronomical events.

The HA of each visits represented in the sample plot below. Other good candidates for plotting in hourglass plots are time use (wide ugr, wide izy, each DDF, lost to weather, lost to instrument failure, etc.), depth, sky brightness, and seeing.

In [ ]:
out_dir = 'tmp'
bundle_group = maf.metric_bundles.MetricBundleGroup(
    bundle_dict=[
        maf.metric_bundles.MetricBundle(
            metric=maf.metrics.MedianMetric("HA"),
            slicer=maf.slicers.VisitIntervalSlicer(),
            constraint="",
            plot_dict={"cmap": plt.get_cmap("coolwarm"), "color_limits": (-4.5, 4.5), 'figsize': (13, 8)},
            plot_funcs=[maf.plots.YearHourglassPlot(day_obs.date.year)],
        )
    ],
    db_con=visits_fname.as_posix(),
    out_dir=out_dir,
)
bundle_group.run_all()
bundle_group.plot_all(closefigs=False)

## DDF Cadence

The y-axis (height of the vertical bars) represents the accumulated effective exposure time, t<sub>eff</sub>, accumulated over all exposures on the field for the night, colored by filter. t<sub>eff</sub> is a measure of survey depth that increases monatonically mith limiting magnitude: t<sub>eff</sub> &prop; 10<sup>&frac45; (m5 - m5<sub>nom</sub>)</sup>. The primary advantage of using t<sub>eff</sub> over a magnitude limit is that it is addative in a coadd: the t<sub>eff</sub> in a properly weighted coadd is the sum of the t<sub>eff</sub>s of the visits that were included in the coadd. As such, it makes sense to "stack" the t<sub>eff</sub> values for a night in a bar plot, while stacking magnitudes is not physically meaningful. (It is called "effective exposure time" because, in the limit of noise domination by sky background photon noise and constant nominal instrument performance and observing conditions, t<sub>eff</sub> is the accumulated exposure time in seconds.) For more details, see [DMTN-296](https://dmtn-296.lsst.io/).

The plot only includes the current DDF "season."

In [ ]:
visits_rp = ResourcePath(visits_fname.as_posix())
visits = schedview.collect.read_opsim(visits_rp, stackers=schedview.collect.visits.DDF_STACKERS)

In [ ]:
ddf_field_names = tuple(ddf_locations().keys())
# Different versions of the schedule include a DD: prefix, or not.
# Catch them all.
ddf_field_names = ddf_field_names + tuple([f"DD:{n}" for n in ddf_field_names])

# Figure out which column has the target names.
target_column_name = "target_name" if "target_name" in visits.columns else "target"
ddf_visits = visits.loc[visits[target_column_name].isin(ddf_field_names)]


In [ ]:
time_window_duration=15 + int(day_obs.mjd - Time('2026-06-30').mjd) % 365
nightly_ddf = schedview.compute.visits.accum_stats_by_target_band_night(ddf_visits)
cadence_plots = schedview.plot.create_cadence_plot(
    nightly_ddf, day_obs.mjd - time_window_duration, day_obs.mjd
)
bokeh.io.show(cadence_plots)